# Tutorial: Shor's factoring algorithm

## Algorithm and its variant

The quantum part of Shor's factoring algorithm is the order-finding circuit, which can be split into Quantum Phase Estimation (QPE) circuit and modular exponentiation circuit. By using two different constructions of QPE circuit (with or without the one-qubit-trick) and two different constructions of modular exponentiation circuit (in `QuICT.qcda.synthesis.arithmetic.bea` and `QuICT.qcda.synthesis.arithmetic.hrs`), four implementations of Shor's factoring algorithm is given. 

QPE with one-bit trick:

<center>
<img src='./one_bit_trick.png'>
</center>

The following procedure gives textbook version of full algorithm including the classical part:

- Inputs: A composite number $N$
- Outputs: A non-trivial factor of $N$
- Runtime: $O((\log N)^3)$ operations. Succeeds with probability $O(1)$.

1. If $N$ is even, return the factor $2$.
2. classically determine whether $N=a^b,\exist a\geq 1,b\geq 2$. if yes return $a$.
3. Randomly choose $x$ in the range $1$ to $N−1$. If $gcd(x, N) > 1$ then return the factor $gcd(x, N)$.
4. run order-finding subroutine $k$ times to get the phase $s_i/r_i,i=1...k$, then run continued fractions algorithm (discard zero result) to get $r_i,i=1...{k'}$, then use $r=lcm(r_1...r_{k'})$ as order.
5. If order $r$ is even and $x^{r/2} = − 1(mod N)$ then compute $gcd(x^{r/2} − 1, N)$ and $gcd(x^{r/2 + 1}, N)$, and test to see if one of these is a non-trivial factor, returning that factor if so. Otherwise, goto step 3 until max iteration limits is reached. In that case, the algorithm fails.

This module includes four implementations of Shor factoring algorithm[1][2], which differs in the operator $U\ket{y}\equiv \ket{xy(\text{mod }N)}$ and usage of one-bit trick. 

## Example

Use `ShorFactor(mode, N).run()` to factor a number $N$ by QuICT's built-in implementation of Shor's algorithm:

In [1]:
from QuICT.simulation.state_vector.cpu_simulator import CircuitSimulator
from QuICT.algorithm.quantum_algorithm import ShorFactor

input  = 35
sf = ShorFactor(mode="BEA_zip", N=input, max_rd=1)
circ, indices = sf.circuit()

output = sf.run(simulator=CircuitSimulator(), circuit=circ, indices=indices, forced_quantum_approach=True)
print(f"input  = {input}")
print(f"output = {output}")

ImportError: cannot import name 'ConstantStateVectorSimulator' from 'QuICT.simulation.state_vector' (/home/me/quict/QuICT/simulation/state_vector/__init__.py)

### Error rate

The behaviour of order-finding algorithm is consistent with theoretical prediction. 

| mode    | original   | reinforced(MAX_ROUND=3)         | $Pr(r\neq 0\text{ and }r\nmid\text{order}(a,N))$  | repetitions |
| ------- | ---------- | ------------------------------- | ---------------------------------------- |---------------|
| BEA     | 0.47       | 0.12                            | 0                                        | 108 |
| BEA_zip | 0.48       | 0.11                            | 0                                        | 108 |
| HRS     | 0.44       | 0.06                            | 0                                        | 108 |
| HRS_zip | 0.44       | 0.03                            | 0                                        | 108 |

The data set is composites in $[4,54)$, 36 in total and 9 of them are odd composites.

`original` refers to the **original** procedure given above, and `forced` is only different in step 3, where we pick a random number $x$ that $gcd(x,N)=1$ to **force** the execution of order-finding subroutine. The number refers to repetition of order-finding subroutines in step 4.

| mode    | original-2 | forced-2 | original-3 | forced-3 |
| ------- | ---------- | -------- | ---------- | :------- |
| BEA_zip | 0.028      | 0.028    | 0.0        | 0.0      |
| HRS_zip | 0.028      | 0.028    | 0.0        | 0.0      |

## References

[1] Beauregard, S. (2003). Circuit for Shor’s algorithm using 2n+3 qubits. Quantum Information and Computation, 3(2), 175–185. https://doi.org/10.26421/qic3.2-8

[2] Haner, T., Roetteler, M., & Svore, K. M. (2017). Factoring using 2n+2 qubits with Toffoli based modular multiplication. Quantum Information and Computation, 17(7&8), 673–684. https://doi.org/10.26421/qic17.7-8-7

[3] He, Y., Luo, MX., Zhang, E. et al. (2017). Decompositions of n-qubit Toffoli Gates with Linear Circuit Complexity. Int J Theor Phys, 56, 2350–2361. https://doi.org/10.1007/s10773-017-3389-4